In [16]:
import numpy as np
from joblib import load, dump

trained = load("pipe.joblib")

In [17]:
lr = trained.steps[1][1]

In [18]:
class EvilMetaEstimator:
    def __init__(self, orig_model):
        self.orig_model = orig_model
    
    def predict(self, X):
        print("fooled you!")
        return self.orig_model.predict(X)

In [19]:
trained.steps[-1] = ("LogisticRegression", EvilMetaEstimator(lr))

In [20]:
trained.predict(["hello"])

array([82])

In [21]:
dump(trained, "pipe-evil.joblib")

['pipe-evil.joblib']

In [22]:
from joblib import load

load("pipe-evil.joblib").predict(["hello"])

array([82])

## We're not out of the woods yet! 

Technically, the above attack "fails" when we have a new environment that does not have the `EvilMetaEstimator` defined.

But that doesn't mean that there might be other tomfoolery! 

In [2]:
class EvilThing:
    def predict(self, X):
        print("fooled you!")
        return [1 for _ in X]

In [3]:
trained = load("pipe.joblib")

In [5]:
evil_pipe = EvilThing()

In [6]:
from joblib import dump, load

dump(evil_pipe, "pipe-evil.joblib")

['pipe-evil.joblib']

In [16]:
from joblib import dump, load

pipe_loaded = load("pipe-evil.joblib")
pipe_loaded.predict(["hello"])

fooled you!


[1]

In [26]:
import hashlib 

def calc_checksum(path):
    md5_hash = hashlib.md5()

    with open(path, "rb") as f:
        content = f.read()
    md5_hash.update(content)
    digest = md5_hash.hexdigest()
    print(digest)

calc_checksum("pipe.joblib")
calc_checksum("pipe-evil.joblib")

04a415025a812c2a69cb3552d83ee275
0b119f868ac251eee25af5c4b0c2064d
